In [3]:
# !apt-get update
# !apt-get install wkhtmltopdf pandoc libfontconfig1 libxrender1 libxext6
# !apt --fix-broken install

In [4]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

# for Google Colab
#from google.colab import userdata
#os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

#print(os.environ['OPENAI_API_KEY'])

True

# Setup

In [6]:
from openai import OpenAI

client = OpenAI()

# retrieve the Assistant
my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# create a thread
my_thread = client.beta.threads.create()


In [7]:
import time

def create_run_and_display_response(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as markdown (with no delimiters) and with no commentary what so ever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"



# Initial File Upload and Prompt

In [8]:
# get the job description from GitHub
import time


file_path = 'temp_files/jobdescription.txt'
# infra engineer example
#!wget -O file_path https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/job-description.md
# medical assistant example
!wget -O 'temp_files/jobdescription.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md

# load it into a variable for later use
with open(file_path, 'r') as file:
    job_description = file.read()

# infra engineer example
# !wget -O companyvalues.txt https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/company-values.md
# medical assistant example
!wget -O 'temp_files/companyvalues.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/about-company.md
# time.sleep(5)


# Upload the file
file_company_values = client.files.create(
  file=open("temp_files/companyvalues.txt", "rb"),
  purpose="assistants"
)

# Upload the file
file_job_description = client.files.create(
  file=open("temp_files/jobdescription.txt", "rb"),
  purpose="assistants"
)
time.sleep(5)



--2023-12-19 20:09:03--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2483 (2.4K) [text/plain]
Saving to: ‘temp_files/jobdescription.txt’

temp_files/jobdescr 100%[===================>]   2.42K  --.-KB/s    in 0s      

2023-12-19 20:09:03 (19.3 MB/s) - ‘temp_files/jobdescription.txt’ saved [2483/2483]

--2023-12-19 20:09:03--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/about-company.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

# Priming the Pump

In [9]:
# Refresh the context to ensure it has read the files
pump_primer = create_run_and_display_response(client,my_thread,my_assistant,"""
            Write a detailed description of the company values from the company values file, using the company name.
            Then, write the job title contained in the job description file.
            Then, create the list of technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description (and no more). Create the list of behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            """, [file_job_description.id, file_company_values.id])
print(pump_primer)


# About Stanford Health Care - University Healthcare Alliance

## Mission
To be the premier community network for the Stanford Health Care System, delivering the best patient experience with the most effective clinical outcomes.  

## Vision  
Healing humanity through science and compassion, one patient at a time.

## Values
- **Innovation**: Embracing innovation, change and continuous improvement to raise the quality of care and shape the future of medicine.
- **Connection**: Cultivating community, connection and care for each other to nurture trust, respect, belonging and shared purpose.
- **Teamwork**: Collaborating across roles, ranks, units, and departments to unlock possibilities for greater impact and fulfillment.
- **Mentorship**: Nurturing and pursuing ongoing professional growth to enjoy impactful and fulfilling careers.
- **Participation**: Freely sharing opinions and valuing diverse backgrounds and contributions to improve decision-making, outcomes, and engagement.
- **Acti

# Create Introduction

In [10]:
introduction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Introduction with the Candidate Profle (only if applicable) and Interview Structure. Be sure to include mention of the competencies in the interview structure.")
print(introduction)

# Interview Playbook for Medical Assistant II

## Introduction

Welcome to the Interview Playbook for the Medical Assistant II position at Stanford Health Care - University Healthcare Alliance. This guide is engineered to equip the hiring team with a values-driven approach for identifying and selecting "A players" who will not only excel technically but also align with the premier community network's culture and values.

The Medical Assistant II role is foundational in providing clinical support to deliver unparalleled patient care. The successful candidate will contribute substantially to our mission of delivering the best patient experience and effective clinical outcomes. During the interview, the objective is to evaluate candidates' technical competencies, such as their clinical acumen and computer proficiency, as well as behavioral competencies that resonate with our values, including innovation, connection, teamwork, mentorship, participation, and decisive action.

### Interview 

# Create Evaluation Criteria

In [11]:
eval_criteria = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

## Evaluation Criteria

This Evaluation Criteria section outlines how to objectively assess the candidate's qualifications for the Medical Assistant II position. Use this guide to keep track of observations and provide comments during the interview process.

| Competency                 | Assessment | Comments |
|----------------------------|------------|----------|
| Clinical Skills            |            |          |
| Communication Skills       |            |          |
| Computer Proficiency       |            |          |
| Organizational Ability     |            |          |
| Policy and Procedure Adherence |        |          |
| Problem-Solving            |            |          |
| Team Collaboration         |            |          |
| Initiative Taking          |            |          |
| Adaptability to Change     |            |          |
| Commitment to Patient Care |            |          |
| Practical Assignment       |            |          |
| Team Interaction        

# Create Technical Assessment

In [12]:
technical_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

Assessing the mastery of clinical procedures and the ability to provide exceptional patient care is critical for a Medical Assistant II. The candidate must demonstrate an advanced understanding of job skills, including the use of C-I-CARE templates for world-class service standards, and patient care protocols.

### Technical Competence 1: Clinical Skills

Clinical skills are the cornerstone of the Medical Assistant II role at Stanford Health Care - University Healthcare Alliance. This competency ensures the candidate has the ability to perform clinical tasks, utilize C-I-CARE standards effectively, and support patient care with the required finesse and expertise.

#### Lead Question

Describe a time when you had to go beyond the standard care procedures to accommodate a patient's needs. How did you ensure that your actions were in line with C-I-CARE standards, and what was the outcome?

#### Probing Questions

- Can you walk me through the steps you take to pre

In [13]:
technical_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Technical Assessment. Don't include the main section header.")
print(technical_assessment_2)

### Technical Competence 2: Computer Proficiency

Computer proficiency is crucial for a Medical Assistant II to manage electronic medical records and utilize various software tools effectively. This competency ensures the candidate can handle the digital aspects of the role, including mastery of Microsoft Office and other relevant systems for efficient operation within a healthcare setting.

#### Lead Question

How do you ensure accuracy and efficiency when managing electronic medical records, and can you give an example of how you've used Microsoft Office to streamline clinical processes?

#### Probing Questions

- Describe a challenging situation you faced with electronic medical records and how you overcame it.
- How have you used Microsoft Excel or other Office tools to support patient care or administrative duties?
- Can you provide an instance where your computer skills significantly improved the outcome of a patient care task?
- What systems have you used in previous roles, and 

In [14]:
technical_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Organizational Ability

Organizational skills are essential for managing the multi-faceted responsibilities of a Medical Assistant II. This competency highlights the candidate’s effectiveness in prioritizing tasks, managing time efficiently, and multi-tasking in a high-paced medical environment.

#### Lead Question

How do you prioritize and organize your daily tasks to ensure the smooth operation of the clinic, and can you provide an example of how your organizational skills have positively impacted patient care?

#### Probing Questions

- Discuss how you manage the demands of multiple tasks while ensuring accuracy and attention to detail.
- Explain your system for keeping track of patient appointments, procedures, and follow-up care.
- Describe a time when you had to adjust your planned schedule due to unexpected circumstances and how you handled it.
- Can you tell us about a time your organizational skills helped to improve a process or procedure in the h

In [15]:
technical_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Policy and Procedure Adherence

Adhering to policies and procedures is necessary to maintain safety and compliance in a healthcare environment. This competency ensures the candidate follows strict guidelines for Basic Life Support (BLS) certification, physical activities, and handling biohazard materials.

#### Lead Question

Discuss how you ensure adherence to healthcare policies and procedures, especially relating to Basic Life Support and the handling of biohazards.

#### Probing Questions

- Can you detail your experience with Basic Life Support in a clinical setting and how you maintain your certification?
- Describe a situation where you had to follow specific procedures for interacting with clinical equipment and machinery.
- Explain the precautions you take when working with biohazards like bloodborne pathogens.
- Share an example of how you've contributed to a safe and compliant work environment through your understanding and application of workplac

In [16]:
technical_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Team Collaboration

Team collaboration is essential for the seamless integration of a Medical Assistant II into the patient care team, ensuring the highest standards of care. This competency is about the ability to work effectively with colleagues, communicate clearly, and contribute to a cohesive team environment.

#### Lead Question

Can you share an experience where your ability to collaborate with a team led to a positive outcome for patient care?

#### Probing Questions

- How do you communicate and coordinate with other team members to ensure a consistent and efficient patient experience?
- Tell us about a time when you had to work closely with other healthcare professionals and how you managed to align your efforts.
- Describe how you approach conflicts or differing opinions within the team.
- What strategies do you use to support and enhance teamwork in a busy healthcare environment?

#### Scenario

You are part of the clinic patient care team at Sta

In [17]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [18]:
behavioral_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

### Behavioral Competence 1: Innovation

Innovation at Stanford Health Care - University Healthcare Alliance involves embracing change and continuously improving to elevate the quality of care. It's about finding and applying new solutions and approaches that will shape the future of healthcare.

#### Lead Question

Tell me about a time when you initiated a change in your work environment that led to an improvement in patient care or clinic operations.

#### Probing Questions

- How do you keep up with the latest advancements in healthcare to incorporate into your role?
- Share an example where you had to be creative to solve a patient care challenge.
- What's the most innovative change you've applied to your work, and what impact did it have?
- How do you approach risk when trying a new method or technology in your work?

#### Scenario

A procedure at Stanford Health Care - University Healthcare Alliance has been identified as in need of improvement to enhance patient satisfaction and

In [19]:
behavioral_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Connection

Connection is about cultivating a sense of community and caring for both patients and colleagues to foster trust, respect, belonging, and shared purpose within Stanford Health Care - University Healthcare Alliance.

#### Lead Question

Can you describe a time when fostering a connection with a patient or a team member made a significant difference in the outcome of treatment or team dynamics?

#### Probing Questions

- How do you build connections with new patients and make them feel at ease?
- Tell us about a time you went the extra mile to support a colleague or improve team morale.
- In what ways do you contribute to creating a positive work environment that reflects our value of connection?
- How do you handle a situation where you perceive a lack of trust or connection with a patient or team member?

#### Scenario

Imagine you’re starting in a new department at Stanford Health Care - University Healthcare Alliance, and you notice a disconne

In [20]:
behavioral_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Teamwork

Teamwork at Stanford Health Care - University Healthcare Alliance is about collaborating across different roles, ranks, and departments to unlock new possibilities and achieving a greater impact. It emphasizes the ability to work cohesively within a team to fulfill the shared mission of the organization.

#### Lead Question

Describe a project or instance where you had to collaborate with a diverse team. How did you ensure effective teamwork and what was the result?

#### Probing Questions

- How do you handle situations where team members have conflicting ideas about patient care?
- Can you give an example of how you have fostered a team environment that is conducive to open discussion and shared decision-making?
- What role do you typically take on in a team setting and why?
- Tell us about a time when you had to depend on other team members to complete a task. How did you manage and facilitate collaboration?

#### Scenario

You’re tasked with c

In [21]:
behavioral_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Mentorship

Mentorship at Stanford Health Care - University Healthcare Alliance encompasses fostering a culture of professional growth to ensure all team members have impactful and fulfilling careers. This value reflects the organization's commitment to nurturing individual development and sharing knowledge.

#### Lead Question

Tell us about a time when you mentored someone or were mentored. How did this experience contribute to your professional growth or the growth of others?

#### Probing Questions

- How do you approach sharing knowledge with new or less experienced colleagues?
- What strategies do you use to encourage and support ongoing learning in your team?
- Describe how mentorship has influenced your career and how you envision it playing a role at Stanford Health Care - University Healthcare Alliance.
- Share an experience when you had to step into a mentor-like role unexpectedly and how you handled it.

#### Scenario

You observed that a newer 

In [22]:
behavioral_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Participation

Participation within Stanford Health Care - University Healthcare Alliance means actively sharing opinions and valuing diverse backgrounds to enhance decision-making, outcomes, and engagement. It illustrates an inclusive culture where diverse perspectives lead to improved patient care.

#### Lead Question

Can you discuss a time when your active participation and sharing of opinions led to better decision-making or results in your professional setting?

#### Probing Questions

- How do you ensure that your voice is heard in a complex healthcare environment?
- Describe a scenario where embracing diverse perspectives within a team made a significant impact.
- In what ways do you encourage others to participate and express their opinions?
- Share an instance where your contribution to a discussion led to a positive change in policy or practice.

#### Scenario

You are involved in a meeting at Stanford Health Care - University Healthcare Alliance

In [23]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [24]:
practical_assignment = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The Practical Assignment for the Medical Assistant II position is designed to evaluate the hands-on capabilities of the candidate in real-world scenarios that they would encounter at Stanford Health Care - University Healthcare Alliance. It should focus on measuring not only their technical skills but also how they embody our values, including innovation, connection, teamwork, mentorship, and participation.

The assignment could involve:

- Conducting a mock patient interview to gather health history and presenting symptoms.
- Demonstrating proper sanitization and setup for a medical procedure.
- Simulating documentation of patient information into our electronic medical records system.

The objective is to observe the candidate in action, assessing their clinical acumen, problem-solving abilities, and how they communicate and interact with patients and staff under simulated conditions.

During the evaluation of the Practical Assignment, interviewers should con

# Create Team Interaction

In [25]:
team_interaction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Interaction

Team interaction is an integral aspect of evaluating the candidate's fit within Stanford Health Care - University Healthcare Alliance. The rationale for including this part in the interview process is to assess how well the candidate could integrate with our existing team, in terms of both skill and alignment with our culture and values such as teamwork, participation, and mentorship.

During this stage, the candidate may engage in:

- Group discussions on relevant healthcare topics to evaluate their communication and interpersonal skills.
- Collaborative problem-solving exercises that test their ability to work in a team and contribute to a shared goal.
- Role-playing scenarios that simulate real workplace challenges assessing the candidate's responses and behavior within a team dynamic.

The candidate's interaction with potential team members should be observed for signs of adaptability, openness to feedback, and the tendency to foster a collaborative environment

# Create Candidate Q&A

In [26]:
candidate_questions = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

During the interview process, candidates often have the opportunity to ask questions about the role, the team, and Stanford Health Care - University Healthcare Alliance. Below are potential questions they might ask, with comprehensive example answers.

1. **What opportunities are there for professional development within the organization?**
   - "At Stanford Health Care - University Healthcare Alliance, we take pride in our commitment to mentorship and continuous improvement. We offer various professional development programs, including ongoing training sessions, workshops, and the chance to work alongside some of the leading professionals in healthcare."

2. **How does the Medical Assistant II role contribute to the overall mission of Stanford Health Care?**
   - "The Medical Assistant II is crucial for delivering excellent patient experiences that align with our mission. By performing clinical tasks, managing patient communications, and assisting with procedures, yo

# Finalize the Playbook HTML

## Create a markdown version

For use in GitHub as an example

In [60]:
playbook_md = (introduction + "\n\n" +
            job_description + "\n\n" +
            eval_criteria + "\n\n" +
            technical_assessment + "\n\n" +
            behavioral_assessment + "\n\n" +
            practical_assignment + "\n\n" +
            candidate_questions + "\n\n" +
            team_interaction + "\n\n" )

# Write playbook to a text file
with open('temp_files/playbook.md', 'w') as file:
    file.write(playbook_md)

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [28]:
import markdown

# Convert Markdown to HTML
html_playbook = markdown.markdown(playbook_md, extensions=['extra','mdx_truly_sane_lists'])

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_introduction = markdown.markdown(introduction, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



 ## Create the style for the document

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [29]:
styles= """

   body {
        font-family: Source Sans Pro;
        font-size: 11pt;
        font-weight: 200;
        font-stretch: 75% 125%;
        color:#0D0D0D;
    }
    @page {
        margin: 0.65in;
        size: Letter;
        @bottom-right {
          content: counter(page);
          font-size: 11pt;
          color: #73706C;
        }
    }
    @page :first {
        @bottom-right {
          content: none;
        }
    }


    h1, h2, h3, h4 {
        font-family: Libre Baskerville;
        margin-top: 1em;
        margin-bottom: 0.5em;
    }
    h1 {
        font-size: 24pt;
        margin-top: 3em;
        page-break-after: avoid;
    }
    .intro h1{
        font-size: 50pt;
        margin-top: 3em;
        page-break-after: always;
    }
    @media print {
      /* Define styles for the cover page */
      .title {
        position: fixed;
        left: 0;
        top: 0;
        width: 100%;
        height: 100%;
        background-color: #F2F2F2; /* Change as needed */
        color: #0D0D0D; /* Change as needed */
        text-align: center;
        padding: 50px;
      }
    }
    .publication-date {
      position: absolute;
      bottom: 10px; /* adjust as needed */
      width: 100%;
      text-align: left;
      font-size: 11pt; /* adjust as needed */
      color: #73706C; /* adjust as needed */
    }


    h2 {
        font-size: 18pt;
        page-break-after: avoid;
    }
    h3 {
        font-size: 14pt;
        page-break-after: avoid;
    }
    h4 {
        font-size: 12pt;
        page-break-after: avoid;
    }
    p {
        margin-bottom: 1em;
    }


    h1, h2, h3, h4 {
        page-break-after: avoid;
    }

    p, table, figure {
        page-break-inside: avoid;
    }
    table {
        width: 100%;
        border-collapse: collapse;
        margin-bottom: 1em;
        page-break-before: avoid;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
        font-size: 10pt;
    }
    th {
        background-color: #f2f2f2;
        font-weight: bold;
    }
    tr:nth-child(even) {{background-color: #f9f9f9;}}

    ul, ol {
        margin-bottom: 1em;
        margin-left: 0;
        padding-left:1.5em;
    }
    li {
        margin-bottom: 0.5em;
        margin-left: 0em;
    }

    .main {
      page-break-before: always;
    }

    .dotted-line {
        border-bottom: 1px dotted black;
        width: 100%;
        margin-top: 20px; /* This ensures spacing between each line */
        margin-bottom: 20px; /* Adjust if needed */
    }

    @media print {
        .dotted-line {
            margin-top: 20px; /* Adjust as needed for printing */
            margin-bottom: 20px; /* Adjust as needed for printing */
        }
    }

    .rating-container {
        display: flex;
        align-items: flex-end;
        margin-top: 20px;
        margin-bottom: 20px;
    }

    .rating-container label {
        margin-right: 10px;
        font-size: 10pt;
        font-family: Source Sans Pro;
    }

    .short-dotted-line {
        border-bottom: 1px dotted black;
        width: 150px; /* Adjust the width as needed */
        margin-left: 5px;
    }}

    @media print {
        .short-dotted-line {
            width: 150px; /* Adjust for printing if needed */
        }
    }
"""

## Create some useful HTML snippets

In [30]:

html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")

## Build the HTML

In [54]:
# Define the HTML template
html_content = rf"""
<meta charset="utf-8">
<html>
<head>
  <link href="https://fonts.googleapis.com/css2?family=Source%20Sans%20Pro:wght@400;700" rel="stylesheet">
  <link href="https://fonts.googleapis.com/css2?family=Libre%20Baskerville:wght@400;700" rel="stylesheet">
  <style>
  {styles}
  </style>
</head>
<body>
    <section class='intro'>
    <div class="publication-date">Published on {publication_date} by the <a href="https://github.com/androb/deliverables/tree/main/interviewplaybook">Interview Playbook Generator</a>!</div>
    {html_introduction}
    </section>
    <section class='main'>
    {html_job_description}
    </section>
    <section class='main'>
    {html_eval_criteria}
    </section>
    <section class='main'>
      {html_technical_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
      {html_technical_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_practical_assignment}
    {html_notes}
    </section>
    <section class='main'>
    {html_team_interaction}
    {html_notes}
    </section>
    <section class='main'>
    {html_candidate_questions}
    </section>

</body>
</html>"""

# Write HTML string to a file
with open('playbook.html', 'w') as file:
    file.write(html_content)



Display the HTML content here so we can review it

In [55]:
# Display the Markdown-formatted text in the notebook
from IPython.display import display, HTML
display(HTML(html_content))

Competency,Assessment,Comments
Clinical Skills,,
Communication Skills,,
Computer Proficiency,,
Organizational Ability,,
Policy and Procedure Adherence,,
Problem-Solving,,
Team Collaboration,,
Initiative Taking,,
Adaptability to Change,,
Commitment to Patient Care,,


# Create the Documents

Create the Word document

In [61]:
# Write HTML string to a file
with open('temp_files/playbook.html', 'w') as file:
    file.write(html_content)

# Convert HTML to Docx
!pandoc -s temp_files/playbook.html -o temp_files/playbook.docx

# from google.colab import files
# files.download('playbook.docx')

Create the PDF

In [57]:
#!python3 --version
#!pango-view --version

#!brew install gobject-introspection
#!brew install gtk+3
#!brew install glade
#!brew install weasyprint

#!python3 -m venv venv
#!source venv/bin/activate
#%pip install cffi weasyprint
#!weasyprint --info

In [62]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML('temp_files/playbook.html').write_pdf('temp_files/playbook.pdf')

# from google.colab import files
# files.download('playbook.pdf')

# Convert HTML to alternative PDF
#!pandoc -s playbook.html --pdf-engine=wkhtmltopdf -o playbook.pdf

# Convert HTML to alternative PDF
#!wkhtmltopdf --page-size Letter --footer-html footer.html --margin-top 15mm --margin-bottom 15mm --margin-left 15mm --margin-right 15mm playbook.html playbook.pdf

# Convert HTML to PDF
##import pdfkit
# pdfkit.from_string(html_content, 'playbook.pdf')
